In [21]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [3]:
# сначала решим задачу проще
# обуимся определять категорию верхнего уровня по объекту
# для этого составим big_categories_id - словарь в котором по новому айди категории лежит 
# список айди старых категорий, с которыми он соотносится
# после изменяются значения в обуччающей  и тестовой выборке


In [7]:
df = read_csv("train.csv")
#df_test = read_csv("test.csv")

In [8]:
df_category = read_csv("category.csv")

In [9]:
# по имени главной категории будут лежать айди вложенных категорий
big_categories = dict()
for cat_id, name in zip(df_category['category_id'],df_category['name']):
    greate_cat = name.split('|')[0]
    if not big_categories.get(greate_cat):
        big_categories[greate_cat] = [cat_id]
    else:
        big_categories[greate_cat].append(cat_id)

big_categories

{'Бытовая электроника': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'Для дома и дачи': [15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29],
 'Личные вещи': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41],
 'Хобби и отдых': [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]}

In [10]:
big_categories_id = dict()
sorted_big_categories = sorted(big_categories)
big_category_id_name = {num: key for num,key in enumerate(sorted_big_categories)}
# по новому айди новой катгории лежит список айди вложенных

In [11]:
for num,key in enumerate(sorted_big_categories):
    big_categories_id[num] = big_categories[key]
big_categories_id

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 1: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 2: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41],
 3: [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]}

In [12]:
reverse_big_categories_id = dict()
for key in big_categories_id:
    for value in big_categories_id[key]:
        reverse_big_categories_id[value] = key

In [13]:
def set_big_category_id(df, reverse_big_categories_id):
    df['big_category_id'] = df['category_id'].apply(reverse_big_categories_id.get)


In [14]:
set_big_category_id(df, reverse_big_categories_id)


In [23]:
# разделение на обучающаю и тестовую
# происходит после преподготовки данных
df_train, df_test = train_test_split(df, test_size = 0.3, random_state = 11) 
def answers(df):
    df_x = df[['item_id', 'title', 'description', 'price']]
    df_y = df[['category_id']]
    df_y_big = df[['big_category_id']]
    return df_x, df_y, df_y_big
df_train_x, df_train_y, df_train_y_big = answers(df_train)
df_test_x, df_test_y, df_test_y_big = answers(df_test)

In [97]:
from nltk.stem.snowball import RussianStemmer, EnglishStemmer
import nltk.corpus as corpus
import re
import nltk; nltk.download('stopwords', quiet=True)
ru_stemmer = RussianStemmer()
en_stemmer = EnglishStemmer()
normalized_stopwords_set = set(map(ru_stemmer.stem, corpus.stopwords.words('russian')))
en_detector = re.compile(r"[A-Za-z]+")
ru_detector = re.compile(r"[А-Яа-я]+")

def normalize(words):
    """
     проверить что слова из обущающей выборке 
     принадлежат не более чем одному языку
    """
    normalized_word_list = []
    for word in words:
        word = word.lower()
        if ru_detector.search(word):
            normalized_word = ru_stemmer.stem(word)
        else:
            normalized_word = en_stemmer.stem(word)
        if normalized_word not in normalized_stopwords_set:
            normalized_word_list.append(normalized_word)
    return normalized_word_list

def get_iter_words(string):
    yield from ( x.group(0) for x in re.finditer(r"[A-Za-zА-Яа-я]+",string))
    


In [108]:
def set_normalize_column(df):
#     df['normalize_title'] =  df['title'].apply(lambda x: normalize(get_iter_words(x)))
    df['normalize_description'] = df['description'].apply(lambda x: normalize(get_iter_words(x)))

In [ ]:
set_normalize_column(df)

In [107]:
df

,item_id,title,description,price,category_id,big_category_id,normalize_title
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,1,[картин]
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,1,"[стул, прессова, кож]"
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,2,"[домашн, мин, бан]"
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,3,"[эксклюзивн, коллекц, книг, трансаэр, подарок]"
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,0,"[ноутбук, aser]"
5,5,Бас гитара invasion bg110,Состояние хорошее. Имеется теплый чехол .,3999.0,50,3,"[бас, гитар, invas, bg]"
6,6,"Смесь ""Грудничок"" г. Зеленодольск",Смесь молочная адаптированная ультрапастеризов...,15.0,41,2,"[сме, грудничок, г, зеленодольск]"
7,7,G-shock,Часы абсолютно новые! с коробкой. Часы Китай...,2500.0,36,2,"[g, shock]"
8,8,"Санатории Белоруссии. - ""Лепельский военный""",Санатории Белоруссии! - «Лепельский военный» ...,1090.0,48,3,"[санатор, белорусс, лепельск, воен]"
9,9,Фотохолст,Фотохолст на подрамнике. 36х58см. Галерейная н...,1250.0,19,1,[фотохолст]


Олег
день
за
днем
То
работает
То
играет
То
Надю
обнимает


'hell'

In [ ]:
#у каждой категории есть
# и у каждождого слова: log(число категорий / число категорий в которых встречается это слово)
#     counter = collections.Counter(normalized_word_list)

def fricvency_words(words):
    """
     забирает список всех
    """
    counter = collections.Counter(words)

In [48]:
df

,item_id,title,description,price,category_id,big_category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,1
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,1
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,2
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,3
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,0
5,5,Бас гитара invasion bg110,Состояние хорошее. Имеется теплый чехол .,3999.0,50,3
6,6,"Смесь ""Грудничок"" г. Зеленодольск",Смесь молочная адаптированная ультрапастеризов...,15.0,41,2
7,7,G-shock,Часы абсолютно новые! с коробкой. Часы Китай...,2500.0,36,2
8,8,"Санатории Белоруссии. - ""Лепельский военный""",Санатории Белоруссии! - «Лепельский военный» ...,1090.0,48,3
9,9,Фотохолст,Фотохолст на подрамнике. 36х58см. Галерейная н...,1250.0,19,1
